In [ ]:
from typing import Dict

import pydriller
import pathlib

In [ ]:
test_modules_path = pathlib.Path("test_submodules")
production_modules_path = pathlib.Path("production_submodules")

In [ ]:
from typing import Literal
from collections import defaultdict

visited_file_names: Dict[str, int] = {}
counters: Dict[Literal["tdd", "same", "non-tdd"], int] = defaultdict(int)

In [ ]:
def is_test_file(filename: str) -> bool:
    return filename.endswith("Test.java")

In [ ]:
def get_code_file_name(filename: str) -> str:
    return filename.replace("Test.java", ".java")

In [ ]:
def get_test_file_name(filename: str) -> str:
    return filename.replace(".java", "Test.java")

In [ ]:
for index, commit in enumerate(pydriller.Repository(str((test_modules_path / "dubboj").absolute())).traverse_commits()):
    # loop from the first commit to the last 
    
    files_in_commit = {modified.filename for modified in commit.modified_files}
    
    for current_file_name in files_in_commit:
        if is_test_file(current_file_name): 
            if current_file_name not in visited_file_names:
                # haven't seen this test before
                code_file_name = get_code_file_name(current_file_name)
                
                if code_file_name in files_in_commit:
                    # the code is in the same commit as the test, same
                    counters["same"] += 1
                else:
                    # the code has not been seen, non-tdd
                    if code_file_name in visited_file_names:
                        # seen code before test
                        counters["non-tdd"] += 1
        else:
            # the current file is a code file
            test_file_name = get_test_file_name(current_file_name)
            
            if test_file_name in visited_file_names:
                # the test file has been seen before
                counters["tdd"] += 1
    
    for file_name in files_in_commit:
        visited_file_names[file_name] = index
        

In [ ]:
print(counters)